# Import packages and database

In [1]:
import pandas as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.segmentation as seg
from biogeme.expressions import Beta, Variable, Derive, exp
from biogeme import models
from biogeme import results as res
from biogeme.expressions import DefineVariable, log
from collections import namedtuple

data_file ='https://raw.githubusercontent.com/GustavePellier/MMOB/main/lpmc19.dat'
LPMC = pd.read_csv(data_file, sep='\t')
LPMC

database = db.Database('LPMC', LPMC)
all_results = {}

# Model 0

We calculate the total public transport duration and the total driving cost

In [2]:
LPMC["dur_pt"]= LPMC["dur_pt_access"] + LPMC["dur_pt_rail"] + LPMC["dur_pt_bus"] + LPMC["dur_pt_int"] 
LPMC["cost_drive"] = LPMC["cost_driving_ccharge"] + LPMC["cost_driving_fuel"]

Some variables are created with the columns that seem to be useful

In [3]:
travel_mode=Variable('travel_mode')

dur_pt_access=Variable('dur_pt_access')
dur_pt_rail=Variable('dur_pt_rail')
dur_pt_bus=Variable('dur_pt_bus')
dur_pt_int=Variable('dur_pt_int')

pt_interchanges=Variable('pt_interchanges')

cost_driving_fuel=Variable('cost_driving_fuel')
cost_driving_ccharge=Variable('cost_driving_ccharge')
cost_drive=Variable('cost_drive')
cost_pt=Variable('cost_transit')

time_walk=Variable('dur_walking')
time_cycle=Variable('dur_cycling')
time_pt=Variable('dur_pt')
time_drive=Variable('dur_driving')

female=Variable('female')
age=Variable('age')

There are 4 different travel mode, we will thus build a model with 4 utility functions, we create 3 alternative specific constant, a generic parameter for travel time and a generic parameter for cost

In [4]:
asc_cycle = Beta('asc_cycle', 0, None, None, 0)
asc_pt = Beta('asc_pt', 0, None, None, 0)
asc_drive = Beta('asc_drive', 0, None, None, 0)
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time = Beta('beta_time', 0, None, None, 0)

Utility functions creation

In [5]:
v_walk_model0= beta_time * time_walk  
v_cycle_model0= asc_cycle + beta_time * time_cycle 
v_pt_model0= asc_pt + beta_time * time_pt + beta_cost * cost_pt
v_drive_model0= asc_drive + beta_time * time_drive + beta_cost * cost_drive

availability of each mode, all available here

In [6]:
av = {1: 1, 2: 1, 3: 1, 4:1}

The estimation results (parameter values, t-tests or p-values, null and final log likelihoods)

In [7]:
V_model0 = {1: v_walk_model0 , 2: v_cycle_model0, 3: v_pt_model0, 4: v_drive_model0}
logprob_model0 = models.loglogit(V_model0, av, travel_mode)
biogeme_model0 = bio.BIOGEME(database, logprob_model0)
biogeme_model0.modelName = 'Model_0'
all_results['Model0'] = biogeme_model0.estimate()
results_generic = biogeme_model0.estimate()
results_generic.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-3.878619,0.107672,-36.022418,0.0
asc_drive,-1.295191,0.080505,-16.088354,0.0
asc_pt,-0.503354,0.054123,-9.300119,0.0
beta_cost,-0.193629,0.013958,-13.871889,0.0
beta_time,-5.495527,0.208596,-26.345342,0.0


In [8]:
res.compile_estimation_results(all_results)

(                                         Model0
 Number of estimated parameters                5
 Sample size                                5000
 Final log likelihood               -4587.818071
 Akaike Information Criterion        9185.636142
 Bayesian Information Criterion      9218.222108
 asc_cycle (t-test)                 -3.88  (-36)
 asc_drive (t-test)                -1.3  (-16.1)
 asc_pt (t-test)                  -0.503  (-9.3)
 beta_cost (t-test)              -0.194  (-13.9)
 beta_time (t-test)                -5.5  (-26.3),
 {'Model0': 'Model0'})

In [9]:
print("Null Loglikelihood : ")
biogeme_model0.calculateNullLoglikelihood(av)

Null Loglikelihood : 


-6931.471805599917

# Model 1

We choose to do a specification for the time attribute, the cost stays generic in this model. Therefore we obtain a beta_time parameter for each transportation mode. 

In [10]:
beta_time_walk = Beta('beta_time_walk', 0, None, None, 0)
beta_time_cycle = Beta('beta_time_cycle', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_drive = Beta('beta_time_drive', 0, None, None, 0)

The utility functions are rewritten with the new parameters :

In [11]:
v_walk_model1= beta_time_walk * time_walk  
v_cycle_model1= asc_cycle + beta_time_cycle * time_cycle 
v_pt_model1= asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model1= asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive

The results are computed (parameter values, t-tests or p-values, null and final log likelihoods)

In [12]:
V_model1 = {1: v_walk_model1 , 2: v_cycle_model1, 3: v_pt_model1, 4: v_drive_model1}
logprob_model1 = models.loglogit(V_model1, av, travel_mode)
biogeme_model1 = bio.BIOGEME(database, logprob_model1)
biogeme_model1.modelName = 'Model_1'
all_results['Model1'] = biogeme_model1.estimate()
results_specific_time = biogeme_model1.estimate()
results_specific_time.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.847226,0.202570,-23.928648,0.0
asc_drive,-1.969875,0.137929,-14.281801,0.0
asc_pt,-2.337198,0.139506,-16.753333,0.0
beta_cost,-0.182009,0.016087,-11.314048,0.0
beta_time_cycle,-5.312452,0.462002,-11.498763,0.0
beta_time_drive,-6.474307,0.378989,-17.083106,0.0
beta_time_pt,-3.538619,0.250656,-14.117426,0.0
beta_time_walk,-8.430774,0.418991,-20.121597,0.0


In [13]:
res.compile_estimation_results(all_results)

(                                         Model0           Model1
 Number of estimated parameters                5                8
 Sample size                                5000             5000
 Final log likelihood               -4587.818071     -4275.496542
 Akaike Information Criterion        9185.636142      8566.993084
 Bayesian Information Criterion      9218.222108       8619.13063
 asc_cycle (t-test)                 -3.88  (-36)   -4.85  (-23.9)
 asc_drive (t-test)                -1.3  (-16.1)   -1.97  (-14.3)
 asc_pt (t-test)                  -0.503  (-9.3)   -2.34  (-16.8)
 beta_cost (t-test)              -0.194  (-13.9)  -0.182  (-11.3)
 beta_time (t-test)                -5.5  (-26.3)                 
 beta_time_cycle (t-test)                          -5.31  (-11.5)
 beta_time_drive (t-test)                          -6.47  (-17.1)
 beta_time_pt (t-test)                             -3.54  (-14.1)
 beta_time_walk (t-test)                           -8.43  (-20.1),
 {'Model0

In [14]:
biogeme_model1.calculateNullLoglikelihood(av)

-6931.471805599917

We compare the 2 models with the likelihood ratio test because model 0 is a reduction of model 1.  

In [15]:
results_specific_time.likelihood_ratio_test(results_generic, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=624.6430583662859, threshold=7.814727903251179)

# Model 2

Using Model 1 as the base model, we choose to study how gender interact with the
ASCs and the alternative attributes. 

Lets start by segmenting the population by gender

In [16]:
gender_segmentation = seg.DiscreteSegmentationTuple(variable=female, mapping={1: 'female', 0: 'male'})

### ASC segmentation

In [17]:
asc_cycle = Beta('asc_cycle', 0, None, None, 0)
segmented_asc_cycle = seg.Segmentation(asc_cycle,[gender_segmentation]).segmented_beta()

asc_pt = Beta('asc_pt', 0, None, None, 0)
segmented_asc_pt = seg.Segmentation(asc_pt,[gender_segmentation]).segmented_beta()

asc_drive = Beta('asc_drive', 0, None, None, 0)
segmented_asc_drive = seg.Segmentation(asc_drive,[gender_segmentation]).segmented_beta()

We redefine the value function of the 4 alternatives, introducing the segmented ASC and the segmented cost attribute

In [18]:
v_walk_model2= beta_time_walk * time_walk  
v_cycle_model2= segmented_asc_cycle + beta_time_cycle * time_cycle 
v_pt_model2= segmented_asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model2= segmented_asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive

We can compute the result of the new model

In [19]:
V_model2 = {1: v_walk_model2 , 2: v_cycle_model2, 3: v_pt_model2, 4: v_drive_model2}
logprob_model2 = models.loglogit(V_model2, av, travel_mode)
biogeme_model2 = bio.BIOGEME(database, logprob_model2)
biogeme_model2.modelName = 'Model_2'
all_results['Model2'] = biogeme_model2.estimate()
results_segmented_gender = biogeme_model2.estimate()
results_segmented_gender.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-5.454900,0.253358,-21.530371,0.000000e+00
asc_cycle_male,1.051961,0.208777,5.038678,4.687574e-07
asc_drive,-1.893639,0.144298,-13.123096,0.000000e+00
asc_drive_male,-0.197932,0.095252,-2.077970,3.771211e-02
asc_pt,-2.208472,0.146368,-15.088460,0.000000e+00
asc_pt_male,-0.333134,0.099553,-3.346293,8.189975e-04
beta_cost,-0.182824,0.016153,-11.317920,0.000000e+00
beta_time_cycle,-5.580983,0.473157,-11.795196,0.000000e+00
beta_time_drive,-6.487836,0.380049,-17.071056,0.000000e+00
beta_time_pt,-3.526934,0.251401,-14.029095,0.000000e+00


In [20]:
biogeme_model2.calculateNullLoglikelihood(av)

-6931.471805599917

### Cost segmentation

And its interaction on the cost attribute in the car and public transport alternative.

In [21]:
beta_cost = Beta('beta_cost', 0, None, None, 0)
segmented_beta_cost = seg.Segmentation(beta_cost, [gender_segmentation]).segmented_beta()

In [22]:
v_walk_model2_2= beta_time_walk * time_walk  
v_cycle_model2_2= asc_cycle + beta_time_cycle * time_cycle 
v_pt_model2_2= asc_pt + beta_time_pt * time_pt + segmented_beta_cost * cost_pt
v_drive_model2_2= asc_drive + beta_time_drive * time_drive + segmented_beta_cost * cost_drive

We can compute the result of the new model

In [23]:
V_model2_2 = {1: v_walk_model2_2 , 2: v_cycle_model2_2, 3: v_pt_model2_2, 4: v_drive_model2_2}
logprob_model2_2 = models.loglogit(V_model2_2, av, travel_mode)
biogeme_model2_2 = bio.BIOGEME(database, logprob_model2_2)
biogeme_model2_2.modelName = 'Model_2_2'
all_results['Model2_2'] = biogeme_model2_2.estimate()
results_segmented_gender_2 = biogeme_model2_2.estimate()
results_segmented_gender_2.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.843794,0.202194,-23.956168,0.000000e+00
asc_drive,-1.971743,0.137733,-14.315671,0.000000e+00
asc_pt,-2.343140,0.139310,-16.819666,0.000000e+00
beta_cost,-0.151591,0.018749,-8.085483,6.661338e-16
beta_cost_male,-0.065394,0.031840,-2.053796,3.999545e-02
beta_time_cycle,-5.348640,0.459248,-11.646521,0.000000e+00
beta_time_drive,-6.483027,0.380414,-17.042044,0.000000e+00
beta_time_pt,-3.536893,0.250704,-14.107825,0.000000e+00
beta_time_walk,-8.435704,0.418411,-20.161286,0.000000e+00


In [24]:
biogeme_model2_2.calculateNullLoglikelihood(av)

-6931.471805599917

### Compiling all results from the different models

In [25]:
res.compile_estimation_results(all_results)

(                                         Model0           Model1  \
 Number of estimated parameters                5                8   
 Sample size                                5000             5000   
 Final log likelihood               -4587.818071     -4275.496542   
 Akaike Information Criterion        9185.636142      8566.993084   
 Bayesian Information Criterion      9218.222108       8619.13063   
 asc_cycle (t-test)                 -3.88  (-36)   -4.85  (-23.9)   
 asc_drive (t-test)                -1.3  (-16.1)   -1.97  (-14.3)   
 asc_pt (t-test)                  -0.503  (-9.3)   -2.34  (-16.8)   
 beta_cost (t-test)              -0.194  (-13.9)  -0.182  (-11.3)   
 beta_time (t-test)                -5.5  (-26.3)                    
 beta_time_cycle (t-test)                          -5.31  (-11.5)   
 beta_time_drive (t-test)                          -6.47  (-17.1)   
 beta_time_pt (t-test)                             -3.54  (-14.1)   
 beta_time_walk (t-test)          

### Likelihood ratio test to check if model_2 and model_2_2 are equivalent or not

In [26]:
results_segmented_gender.likelihood_ratio_test(results_segmented_gender_2, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=55.92757901896948, threshold=5.991464547107979)

Model_2 is preferred because they are not equivalent. The BIC and AIC are smaller and the final likelihood is bigger for model 2

### Comparison with model 1

In [27]:
results_segmented_gender.likelihood_ratio_test(results_specific_time, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=60.51262179189689, threshold=7.814727903251179)

Model_2 is preferred to Model_1 because they are not equivalent. The BIC and AIC are smaller and the final likelihood is bigger for model 2.

# Model 3 - Thomas

In [28]:
# Repartir du model segmenté, Modèle 2

In [29]:
#GENDER segmentation
gender_segmentation_3bis = seg.DiscreteSegmentationTuple(variable=female, mapping={1: 'female', 0: 'male'})

In [30]:
asc_cycle_3bis = Beta('asc_cycle_3bis', 0, None, None, 0)
segmented_asc_cycle_3bis = seg.Segmentation(asc_cycle,[gender_segmentation]).segmented_beta()

asc_pt_3bis = Beta('asc_pt_3bis', 0, None, None, 0)
segmented_asc_pt_3bis = seg.Segmentation(asc_pt,[gender_segmentation]).segmented_beta()

asc_drive_3bis = Beta('asc_drive_3bis', 0, None, None, 0)
segmented_asc_drive_3bis = seg.Segmentation(asc_drive_3bis,[gender_segmentation_3bis]).segmented_beta()

In [31]:
beta_time_walk_3bis = Beta('beta_time_walk_3bis', 0, None, None, 0)
beta_time_cycle_3bis = Beta('beta_time_cycle_3bis', 0, None, None, 0)

beta_time_pt_3bis = Beta('beta_time_pt_3bis', 0, None, None, 0)
beta_time_pt_3bis_squarred = Beta('beta_time_pt_3bis_squarred', 0, None, None, 0)

beta_time_drive_3bis = Beta('beta_time_drive_3bis', 0, None, None, 0)
beta_cost_3bis = Beta('beta_cost_3bis', 0, None, None, 0)


v_walk_model_3bis= beta_time_walk_3bis * time_walk  

v_cycle_model_3bis= segmented_asc_cycle_3bis + beta_time_cycle_3bis * time_cycle 

v_pt_model_3bis= segmented_asc_pt_3bis + beta_time_pt_3bis * time_pt + beta_time_pt_3bis_squarred *time_pt*time_pt + beta_cost_3bis * cost_pt

v_drive_model_3bis= segmented_asc_drive_3bis + beta_time_drive_3bis * time_drive + beta_cost_3bis * cost_drive

In [32]:
V_model_3bis = {1: v_walk_model_3bis , 2: v_cycle_model_3bis, 3: v_pt_model_3bis, 4: v_drive_model_3bis}
logprob_model_3bis = models.loglogit(V_model_3bis, av, travel_mode)
biogeme_model_3bis = bio.BIOGEME(database, logprob_model_3bis)
biogeme_model_3bis.modelName = 'Model_3bis'
all_results['Model_3bis'] = biogeme_model_3bis.estimate()
results_segmented_gender_3bis = biogeme_model_3bis.estimate()
results_segmented_gender_3bis.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-5.373633,0.250689,-21.435415,0.000000e+00
asc_cycle_male,1.052224,0.208454,5.047748,4.470478e-07
asc_drive_3bis,-1.802140,0.145490,-12.386676,0.000000e+00
asc_drive_3bis_male,-0.198042,0.094447,-2.096845,3.600732e-02
asc_pt,-2.490305,0.160124,-15.552319,0.000000e+00
asc_pt_male,-0.339756,0.101136,-3.359391,7.811443e-04
beta_cost_3bis,-0.180475,0.016392,-11.010049,0.000000e+00
beta_time_cycle_3bis,-5.573343,0.458404,-12.158134,0.000000e+00
beta_time_drive_3bis,-6.562530,0.388238,-16.903348,0.000000e+00
beta_time_pt_3bis,-1.983146,0.470332,-4.216483,2.481418e-05


In [33]:
#COST segmentation

In [34]:
beta_cost_33 = Beta('beta_cost_33', 0, None, None, 0)
segmented_beta_cost_33 = seg.Segmentation(beta_cost_33, [gender_segmentation_3bis]).segmented_beta()

In [35]:
beta_time_pt_33_squarred = Beta('beta_time_pt_33_squarred', 0, None, None, 0)

v_walk_model_33= beta_time_walk * time_walk  

v_cycle_model_33= asc_cycle + beta_time_cycle * time_cycle 

v_pt_model_33= asc_pt + beta_time_pt * time_pt + beta_time_pt_33_squarred * time_pt * time_pt + segmented_beta_cost * cost_pt

v_drive_model_33= asc_drive + beta_time_drive * time_drive + segmented_beta_cost * cost_drive

In [36]:
V_model_33 = {1: v_walk_model_33 , 2: v_cycle_model_33, 3: v_pt_model_33, 4: v_drive_model_33}
logprob_model_33 = models.loglogit(V_model_33, av, travel_mode)
biogeme_model_33 = bio.BIOGEME(database, logprob_model_33)
biogeme_model_33.modelName = 'Model__33'
all_results['Model_33'] = biogeme_model_33.estimate()
results_segmented_gender_33 = biogeme_model_33.estimate()
results_segmented_gender_33.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.761736,0.199493,-23.869143,0.000000e+00
asc_drive,-1.881527,0.139072,-13.529112,0.000000e+00
asc_pt,-2.623410,0.154065,-17.027935,0.000000e+00
beta_cost,-0.149036,0.018925,-7.875277,3.330669e-15
beta_cost_male,-0.065963,0.032234,-2.046417,4.071537e-02
beta_time_cycle,-5.345714,0.445822,-11.990688,0.000000e+00
beta_time_drive,-6.555686,0.388305,-16.882845,0.000000e+00
beta_time_pt,-2.013847,0.470794,-4.277553,1.889591e-05
beta_time_pt_33_squarred,-1.219341,0.352244,-3.461637,5.369008e-04
beta_time_walk,-8.288615,0.418773,-19.792639,0.000000e+00


### Comparison model_3bis and model_33

In [37]:


results_segmented_gender_3bis.likelihood_ratio_test(results_segmented_gender_33, 0.05)


LRTuple(message='H0 can be rejected at level 5.0%', statistic=56.37685201148088, threshold=5.991464547107979)

In [38]:
print(results_segmented_gender_3bis.printGeneralStatistics())

Number of estimated parameters:	12
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4236.589
Final log likelihood:	-4236.589
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00283
Akaike Information Criterion:	8497.178
Bayesian Information Criterion:	8575.385
Final gradient norm:	2.8387E-04
Nbr of threads:	4



In [39]:
print(results_segmented_gender_33.printGeneralStatistics())

Number of estimated parameters:	10
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4264.778
Final log likelihood:	-4264.778
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00234
Akaike Information Criterion:	8549.555
Bayesian Information Criterion:	8614.727
Final gradient norm:	2.1793E-04
Nbr of threads:	4



### Preferred model is model_3bis

The model_3bis is preferred to model_33 as its log likelihood is greater and the AIC and BIC are smaller as well for this model.

### Comparison model_3bis and model_2

In [40]:
results_segmented_gender_3bis.likelihood_ratio_test(results_segmented_gender, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=17.302101690711424, threshold=3.841458820694124)

In [41]:
print(results_segmented_gender.printGeneralStatistics())

Number of estimated parameters:	11
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4245.24
Final log likelihood:	-4245.24
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00259
Akaike Information Criterion:	8512.48
Bayesian Information Criterion:	8584.17
Final gradient norm:	2.1215E-02
Nbr of threads:	4



We notice that the Final loglikelyhood as well as AIC & BIC are lower for the model_3bis than for model_2.
As those two model are not equivalent at the 5% significance level, we can conclude that the model 3 bis is preferred

### Version ancienne du power series 

In [42]:
# Power Series transformation on 'pt_dur'.

asc_cycle_3 = Beta('asc_cycle_3', 0, None, None, 0)

asc_pt_3 = Beta('asc_pt_3', 0, None, None, 0)

asc_drive_3 = Beta('asc_drive_3', 0, None, None, 0)


beta_time_walk_3 = Beta('beta_time_walk_3', 0, None, None, 0)
beta_time_cycle_3 = Beta('beta_time_cycle_3', 0, None, None, 0)

beta_time_pt_3 = Beta('beta_time_pt_3', 0, None, None, 0)
beta_time_pt_3_squarred = Beta('beta_time_pt_3_squarred', 0, None, None, 0)

beta_time_drive_3 = Beta('beta_time_drive_3', 0, None, None, 0)
beta_cost_3 = Beta('beta_cost_3', 0, None, None, 0)



v_walk_model_3= beta_time_walk_3 * time_walk 

v_cycle_model_3= asc_cycle_3 + beta_time_cycle_3 * time_cycle 

v_pt_model_3= asc_pt_3 + beta_time_pt_3 * time_pt +beta_time_pt_3_squarred * time_pt * time_pt + beta_cost * cost_pt

v_drive_model_3= asc_drive_3 + beta_time_drive_3 * time_drive + beta_cost_3 * cost_drive

In [43]:
V_model_3 = {1: v_walk_model_3 , 2: v_cycle_model_3, 3: v_pt_model_3, 4: v_drive_model_3}

logprob_model_3 = models.loglogit(V_model_3, av, travel_mode)

biogeme_model_3 = bio.BIOGEME(database, logprob_model_3)

biogeme_model_3.modelName = 'Model_3'

all_results['Model_3'] = biogeme_model_3.estimate()

results_power_3 = biogeme_model_3.estimate()

results_power_3.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle_3,-4.763434,0.199847,-23.835398,0.000000e+00
asc_drive_3,-1.873747,0.139320,-13.449255,0.000000e+00
asc_pt_3,-2.601264,0.155172,-16.763702,0.000000e+00
beta_cost,-0.210770,0.031140,-6.768429,1.301870e-11
beta_cost_3,-0.168058,0.017600,-9.548949,0.000000e+00
beta_time_cycle_3,-5.431876,0.464148,-11.702905,0.000000e+00
beta_time_drive_3,-6.753132,0.442152,-15.273340,0.000000e+00
beta_time_pt_3,-2.024756,0.471819,-4.291381,1.775656e-05
beta_time_pt_3_squarred,-1.228551,0.352701,-3.483264,4.953389e-04
beta_time_walk_3,-8.319325,0.421317,-19.746000,0.000000e+00


In [44]:
#Calculate Null Log likelihood

biogeme_model_3.calculateNullLoglikelihood(av)

-6931.471805599917

In [45]:
#Comparison to model 1

results_power_3.likelihood_ratio_test(results_specific_time, 0.05)




LRTuple(message='H0 can be rejected at level 5.0%', statistic=18.42252238043693, threshold=5.991464547107979)

The model 3 is not preferred compare to model 1.

-Is it due to the non linear transformation that is not appropriate ? Should we use boxplot instead ? --> Yes we should 

-Is it stated 'non-linear transfronation on one of the variables' so I just took public transport time, should I apply to the other time ? --> I think but I will ask

-Should the non linear transformation be applied to an other time variable like car ? Yes I think but I will ask

-Is the loglikelyhood ratio test still appropirate ? --> Maybe we should use a cox-text instead 

Remarks: I think that the non-linear transformation is not done on beta but rather on variables such as time, cost, etc.!


# Model 3 - Salomé

In [46]:

lambda_boxcox = Beta('lambda_boxcox', 1, None, None, 0) #vu dans le modèle 01-logit_airline_solution
boxcox_time_walk = models.boxcox(time_walk, lambda_boxcox)
boxcox_time_cycle = models.boxcox(time_cycle, lambda_boxcox)
boxcox_time_pt = models.boxcox(time_pt, lambda_boxcox)
boxcox_time_drive = models.boxcox(time_drive , lambda_boxcox)
beta_elapsed_time_walk = Beta('beta_elapsed_walk', 0, None, None, 0)
beta_elapsed_time_cycle = Beta('beta_elapsed_cycle', 0, None, None, 0)
beta_elapsed_time_pt = Beta('beta_elapsed_time_pt', 0, None, None, 0)
beta_elapsed_time_drive = Beta('beta_elapsed_time_drive', 0, None, None, 0)


v_walk_model2= beta_time_walk * time_walk  
v_cycle_model2= segmented_asc_cycle + beta_time_cycle * time_cycle 
v_pt_model2= segmented_asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model2= segmented_asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive


v_walk_model3 = beta_elapsed_time_walk * boxcox_time_walk
v_cycle_model3  = segmented_asc_cycle + beta_elapsed_time_cycle * boxcox_time_cycle 
v_pt_model3  = segmented_asc_pt + beta_elapsed_time_pt * boxcox_time_pt   + beta_cost * cost_pt 
v_drive_model3  = segmented_asc_drive + beta_elapsed_time_drive * boxcox_time_drive + beta_cost * cost_drive 

It is advised to set the bounds on parameter lambda_boxcox. A value of -10 and 10 should be appropriate: Beta("lambda_boxcox", 1, -10, 10, 0)
It is advised to set the bounds on parameter lambda_boxcox. A value of -10 and 10 should be appropriate: Beta("lambda_boxcox", 1, -10, 10, 0)
It is advised to set the bounds on parameter lambda_boxcox. A value of -10 and 10 should be appropriate: Beta("lambda_boxcox", 1, -10, 10, 0)
It is advised to set the bounds on parameter lambda_boxcox. A value of -10 and 10 should be appropriate: Beta("lambda_boxcox", 1, -10, 10, 0)


In [47]:
V_model3 = {1: v_walk_model3 , 2: v_cycle_model3, 3: v_pt_model3, 4: v_drive_model3}
logprob_model3 = models.loglogit(V_model3, av, travel_mode)
biogeme_model3 = bio.BIOGEME(database, logprob_model3)
biogeme_model3.modelName = 'Model_3'
all_results['Model_3'] = biogeme_model3.estimate()
results_model3 = biogeme_model3.estimate()
results_model3.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-3.218606,0.303923,-10.590195,0.000000e+00
asc_cycle_male,1.053227,0.210050,5.014181,5.325996e-07
asc_drive,0.068991,0.217652,0.316979,7.512599e-01
asc_drive_male,-0.226072,0.101484,-2.227663,2.590298e-02
asc_pt,1.844840,0.170762,10.803555,0.000000e+00
asc_pt_male,-0.375195,0.107310,-3.496354,4.716630e-04
beta_cost,-0.174891,0.015669,-11.161353,0.000000e+00
beta_elapsed_cycle,-3.318160,0.268402,-12.362656,0.000000e+00
beta_elapsed_time_drive,-3.301174,0.265809,-12.419363,0.000000e+00
beta_elapsed_time_pt,-2.637911,0.182667,-14.441099,0.000000e+00


In [48]:
res.compileEstimationResults(all_results)

The syntax "compileEstimationResults" is deprecated and is replaced by the syntax "compile_estimation_results".


(                                              Model0           Model1  \
 Number of estimated parameters                     5                8   
 Sample size                                     5000             5000   
 Final log likelihood                    -4587.818071     -4275.496542   
 Akaike Information Criterion             9185.636142      8566.993084   
 Bayesian Information Criterion           9218.222108       8619.13063   
 asc_cycle (t-test)                      -3.88  (-36)   -4.85  (-23.9)   
 asc_drive (t-test)                     -1.3  (-16.1)   -1.97  (-14.3)   
 asc_pt (t-test)                       -0.503  (-9.3)   -2.34  (-16.8)   
 beta_cost (t-test)                   -0.194  (-13.9)  -0.182  (-11.3)   
 beta_time (t-test)                     -5.5  (-26.3)                    
 beta_time_cycle (t-test)                               -5.31  (-11.5)   
 beta_time_drive (t-test)                               -6.47  (-17.1)   
 beta_time_pt (t-test)                

In [49]:
print("Null Loglikelihood : ")
biogeme_model3.calculateNullLoglikelihood(av)

Null Loglikelihood : 


-6931.471805599917

In [50]:
results_model3.likelihood_ratio_test(results_segmented_gender, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=175.98772982144692, threshold=3.841458820694124)

In [51]:
print(results_model3.printGeneralStatistics())

Number of estimated parameters:	12
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4157.246
Final log likelihood:	-4157.246
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00289
Akaike Information Criterion:	8338.493
Bayesian Information Criterion:	8416.699
Final gradient norm:	8.5054E-04
Nbr of threads:	4



To compare Model 2 and Model 3 I think we should use a Cox test as Model 3 is not a a linear restriction of Model 2, to do that we create a composite model C such that both models 1 and 2 are restricted cases of model C.


In [52]:
v_walk_model2= beta_time_walk * time_walk  
v_cycle_model2= segmented_asc_cycle + beta_time_cycle * time_cycle 
v_pt_model2= segmented_asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model2= segmented_asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive


v_walk_model3 = beta_elapsed_time_walk * boxcox_time_walk
v_cycle_model3  = segmented_asc_cycle + beta_elapsed_time_cycle * boxcox_time_cycle 
v_pt_model3  = segmented_asc_pt + beta_elapsed_time_pt * boxcox_time_pt   + beta_cost * cost_pt 
v_drive_model3  = segmented_asc_drive + beta_elapsed_time_drive * boxcox_time_drive + beta_cost * cost_drive 


v_walk_model_generic = beta_time_walk * time_walk + beta_elapsed_time_walk * boxcox_time_walk
v_cycle_model_generic = segmented_asc_cycle + beta_time_cycle * time_cycle +  beta_elapsed_time_cycle * boxcox_time_cycle 
v_pt_model_generic = segmented_asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt + beta_elapsed_time_pt * boxcox_time_pt 
v_drive_model_generic = segmented_asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive + beta_elapsed_time_drive * boxcox_time_drive

In [53]:
V_model_generic = {1: v_walk_model_generic , 2: v_cycle_model_generic, 3: v_pt_model_generic, 4: v_drive_model_generic}
logprob_model_generic = models.loglogit(V_model_generic, av, travel_mode)
biogeme_model_generic = bio.BIOGEME(database, logprob_model_generic)
biogeme_model_generic.modelName = 'Model_generic'
all_results['Model_generic'] = biogeme_model_generic.estimate()
results_model_generic = biogeme_model_generic.estimate()
results_model_generic.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-2.699049,1.213496,-2.224193,2.613544e-02
asc_cycle_male,1.056065,0.209816,5.033294,4.821232e-07
asc_drive,0.708587,0.898890,0.788291,4.305263e-01
asc_drive_male,-0.224706,0.101183,-2.220780,2.636590e-02
asc_pt,5.566448,1.723189,3.230319,1.236523e-03
asc_pt_male,-0.369762,0.108178,-3.418078,6.306506e-04
beta_cost,-0.170998,0.015858,-10.783394,0.000000e+00
beta_elapsed_cycle,-5.506688,2.108741,-2.611363,9.018221e-03
beta_elapsed_time_drive,-5.415447,2.406850,-2.250015,2.444802e-02
beta_elapsed_time_pt,-2.447031,1.033187,-2.368429,1.786379e-02


In [54]:
res.compileEstimationResults(all_results)

The syntax "compileEstimationResults" is deprecated and is replaced by the syntax "compile_estimation_results".


(                                              Model0           Model1  \
 Number of estimated parameters                     5                8   
 Sample size                                     5000             5000   
 Final log likelihood                    -4587.818071     -4275.496542   
 Akaike Information Criterion             9185.636142      8566.993084   
 Bayesian Information Criterion           9218.222108       8619.13063   
 asc_cycle (t-test)                      -3.88  (-36)   -4.85  (-23.9)   
 asc_drive (t-test)                     -1.3  (-16.1)   -1.97  (-14.3)   
 asc_pt (t-test)                       -0.503  (-9.3)   -2.34  (-16.8)   
 beta_cost (t-test)                   -0.194  (-13.9)  -0.182  (-11.3)   
 beta_time (t-test)                     -5.5  (-26.3)                    
 beta_time_cycle (t-test)                               -5.31  (-11.5)   
 beta_time_drive (t-test)                               -6.47  (-17.1)   
 beta_time_pt (t-test)                

In [55]:
model_base=biogeme_model2
results_base=model_base.estimate()
results_model_generic.likelihood_ratio_test(results_base, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=191.55170486268798, threshold=11.070497693516351)

In [56]:
model_base=biogeme_model3
results_base=model_base.estimate()
results_model_generic.likelihood_ratio_test(results_base, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=15.563975041241065, threshold=9.487729036781154)

## Model 4

We chose to separate the modes by nesting walking and cycling as "soft modes" and keeping driving and PT separeted. 

In [57]:
model_base=biogeme_model3
V_nested=V_model3
#create the nests
Mu_Drive =Beta("Mu_Drive",1,0,None,0)
Mu_PT =Beta("Mu_PT",1,0,None,0)
Mu_SM =Beta("Mu_SoftModes",1,0,None,0)
drive= Mu_Drive,[4]
PT=Mu_PT, [3]
SM=Mu_SM, [1,2]
nests_modes=drive, PT, SM

In [58]:
logprob_model_nested = models.lognested(V_nested, av, nests_modes, travel_mode)
biogeme_model_nested = bio.BIOGEME(database, logprob_model_nested)
biogeme_model_nested.modelName = 'nestmodes'
all_results['Nestmodes'] = biogeme_model_nested.estimate()
results_model_nested = biogeme_model_nested.estimate()
results_model_nested.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
Mu_Drive,1.000000,1.797693e+308,5.562685e-309,1.000000e+00
Mu_PT,1.000000,1.797693e+308,5.562685e-309,1.000000e+00
Mu_SoftModes,0.603680,7.647242e-02,7.894089e+00,2.886580e-15
asc_cycle,-2.955126,3.748133e-01,-7.884260e+00,3.108624e-15
asc_cycle_male,1.574252,3.494972e-01,4.504333e+00,6.658162e-06
asc_drive,0.294558,2.368810e-01,1.243484e+00,2.136895e-01
asc_drive_male,-0.147821,1.109199e-01,-1.332681e+00,1.826365e-01
asc_pt,2.181106,1.969323e-01,1.107541e+01,0.000000e+00
asc_pt_male,-0.290546,1.165381e-01,-2.493145e+00,1.266173e-02
beta_cost,-0.171546,1.569916e-02,-1.092708e+01,0.000000e+00


The model is rejected as Mu soft modes in < 1

We try another model : vehicles (bikes, cars, PT) and not vehicle (walk) 

In [59]:
Mu_vehicle=Beta("Mu_vehicle",1,0,None,0)
Mu_not_vehicle=Beta("Mu_not_vehicle",1,0,None,0)
vehicle= Mu_vehicle,[2,3,4]
not_vehicle=Mu_not_vehicle,[1]
nests_vehicle=vehicle, not_vehicle


In [60]:
logprob_model_nestedvehicle = models.lognested(V_nested, av, nests_vehicle, travel_mode)
biogeme_model_nestedvehicle = bio.BIOGEME(database, logprob_model_nestedvehicle)
biogeme_model_nestedvehicle.modelName = 'Model_nested_vehicle'
results_model_nestedvehicle = biogeme_model_nestedvehicle.estimate()
all_results['Model_nested_vehicle'] = results_model_nestedvehicle
table_nestedvehicle=results_model_nestedvehicle.getEstimatedParameters()
table_nestedvehicle

,Value,Rob. Std err,Rob. t-test,Rob. p-value
Mu_not_vehicle,1.000000,1.797693e+308,5.562685e-309,1.000000e+00
Mu_vehicle,1.141124,1.709579e-01,6.674886e+00,2.474243e-11
asc_cycle,-2.355505,9.595256e-01,-2.454864e+00,1.409377e-02
asc_cycle_male,0.900627,2.489309e-01,3.617980e+00,2.969115e-04
asc_drive,0.520153,5.210114e-01,9.983518e-01,3.181088e-01
asc_drive_male,-0.221861,1.000834e-01,-2.216765e+00,2.663913e-02
asc_pt,2.075649,2.937994e-01,7.064852e+00,1.607825e-12
asc_pt_male,-0.352523,1.071189e-01,-3.290949e+00,9.984990e-04
beta_cost,-0.153388,2.784711e-02,-5.508226e+00,3.624683e-08
beta_elapsed_cycle,-2.882927,5.249758e-01,-5.491543e+00,3.984378e-08


In [61]:
res.compileEstimationResults(all_results)

The syntax "compileEstimationResults" is deprecated and is replaced by the syntax "compile_estimation_results".


(                                              Model0           Model1  \
 Number of estimated parameters                     5                8   
 Sample size                                     5000             5000   
 Final log likelihood                    -4587.818071     -4275.496542   
 Akaike Information Criterion             9185.636142      8566.993084   
 Bayesian Information Criterion           9218.222108       8619.13063   
 asc_cycle (t-test)                      -3.88  (-36)   -4.85  (-23.9)   
 asc_drive (t-test)                     -1.3  (-16.1)   -1.97  (-14.3)   
 asc_pt (t-test)                       -0.503  (-9.3)   -2.34  (-16.8)   
 beta_cost (t-test)                   -0.194  (-13.9)  -0.182  (-11.3)   
 beta_time (t-test)                     -5.5  (-26.3)                    
 beta_time_cycle (t-test)                               -5.31  (-11.5)   
 beta_time_drive (t-test)                               -6.47  (-17.1)   
 beta_time_pt (t-test)                

In [62]:
results_base=model_base.estimate()
results_model_nestedvehicle.likelihood_ratio_test(results_base, 0.05)

LRTuple(message='H0 cannot be rejected at level 5.0%', statistic=0.904556685625721, threshold=5.991464547107979)

So the model pref keeps being Model_3 (Box-cox transformation)

## Market Shares

The census data report the following number $N_g$ of individuals in each segment $g$.

In [63]:
census = {
    'male_45_more':  1379198,
    'male_45_less':  2926408,
    'female_45_more':   1519948,
    'female_45_less':  2841376,
}

The total size $N$ of the population is the sum over all segments.

In [64]:
total = sum(census.values())
total

8666930

Identify each segment in the database.

In [65]:
filters = {
    'male_45_more': (LPMC.age >=45) & (LPMC.female == 0),
    'male_45_less': (LPMC.age < 45) & (LPMC.female == 0),
    'female_45_more': (LPMC.age >= 45) & (LPMC.female == 1),
    'female_45_less': (LPMC.age < 45) & (LPMC.female == 1),
}

We count the number 
$S_g$ of individuals in each segment $g$ in the sample.

In [66]:

sample_segments = {
    k: v.sum() for k, v in filters.items()
}
sample_segments

{'male_45_more': 903,
 'male_45_less': 1436,
 'female_45_more': 997,
 'female_45_less': 1664}

We check the total sample size $S$.

In [67]:
total_sample = sum(sample_segments.values())
total_sample

5000

The weight $w_g$ associated with segment $g$ is defined as
$$
w_g = \frac{N_g}{N}\frac{S}{S_g}.
$$

In [68]:
weights = {
    k: census[k] * total_sample / (v * total) 
    for k, v in sample_segments.items()
}
weights

{'male_45_more': 0.881137295471826,
 'male_45_less': 1.17566922738916,
 'female_45_more': 0.8795049485193039,
 'female_45_less': 0.9850995060002171}

We insert the weight as a new column in the database.

In [69]:
for k, f in filters.items():
    LPMC.loc[f, 'Weight'] = weights[k] 

We check that the sum of the weights is the sample size.

In [70]:
sum_weights = LPMC['Weight'].sum()
sum_weights

5000.0

It is equal.

#### Perform sample enumeration

The goal is to estimate predicted shares using the formula :
    $$Ŵ(i) = \frac{1}{S}\sum_{n=1}^{S}w_n*P(i|x_n; θ)$$

First we need to obtain $P(i|x_n; θ)$ for each option $i$

Using Model 3 ( the preferred model so far ) and the logit model $$ P(i|x_n; θ)= \frac{exp(V_{in})}{\sum_{j \in C}{exp(V_{jn})}}$$ where $\mu$ is normalized to 1.

In [71]:
prob_walk =  exp(v_walk_model3) /( exp(v_walk_model3)+ exp(v_drive_model3)+exp(v_pt_model3)+exp(v_cycle_model3))
prob_drive = exp(v_drive_model3) /( exp(v_walk_model3)+ exp(v_drive_model3)+exp(v_pt_model3)+exp(v_cycle_model3))
prob_cycle = exp(v_cycle_model3) /( exp(v_walk_model3)+ exp(v_drive_model3)+exp(v_pt_model3)+exp(v_cycle_model3))
prob_pt =  exp(v_pt_model3) /( exp(v_walk_model3)+ exp(v_drive_model3)+exp(v_pt_model3)+exp(v_cycle_model3))

We calculate the choice probability of each alternative for each observation. We include the weight in the simulation results.

In [72]:
Weight = Variable('Weight')

In [73]:
simulate = {
    'Weight': Weight,
    'Prob. walk': prob_walk,
    'Prob. drive': prob_drive,
    'Prob. cycle': prob_cycle,
    'Prob. pt': prob_pt,    
}

In [74]:
database = db.Database('LPMC',  LPMC)
biosim = bio.BIOGEME(database, simulate)
simulated_values = biosim.simulate(results_model3.getBetaValues())

In [75]:
simulated_values


,Weight,Prob. walk,Prob. drive,Prob. cycle,Prob. pt
0,0.879505,0.001361,0.231611,0.010644,0.756383
1,0.881137,0.000098,0.199559,0.020164,0.780178
2,1.175669,0.029810,0.441931,0.037850,0.490409
3,1.175669,0.586454,0.203987,0.027478,0.182081
4,0.879505,0.413058,0.343854,0.017955,0.225134
...,...,...,...,...,...
4995,1.175669,0.449571,0.387903,0.044240,0.118286
4996,1.175669,0.000237,0.859191,0.010705,0.129867
4997,1.175669,0.105456,0.717433,0.063042,0.114069
4998,0.879505,0.123392,0.532410,0.026802,0.317396


Market shares are calculated using the weighted mean of the
individual probabilities.

In [76]:
simulated_values['Weighted walk'] = (
    simulated_values['Weight'] * 
    simulated_values['Prob. walk']
)
simulated_values['Weighted drive'] = (
    simulated_values['Weight'] * 
    simulated_values['Prob. drive']
)
simulated_values['Weighted cycle'] = (
    simulated_values['Weight'] * 
    simulated_values['Prob. cycle']
)
simulated_values['Weighted pt'] = (
    simulated_values['Weight'] * 
    simulated_values['Prob. pt']
)

In [77]:
market_share_walk = simulated_values['Weighted walk'].mean()
print(f'Market share for walk: {100*market_share_walk:.1f}%')
market_share_drive = simulated_values['Weighted drive'].mean()
print(f'Market share for drive: {100*market_share_drive:.1f}%')
market_share_cycle = simulated_values['Weighted cycle'].mean()
print(f'Market share for cycle: {100*market_share_cycle:.1f}%')
market_share_pt = simulated_values['Weighted pt'].mean()
print(f'Market share for pt: {100*market_share_pt:.1f}%')

Market share for walk: 17.8%
Market share for drive: 43.0%
Market share for cycle: 3.0%
Market share for pt: 36.2%


#### Perform sample enumeration

In order to calculate confidence intervals, we need to re-estimate the parameters using bootstrapping.

In [78]:
biogeme_model3 = bio.BIOGEME(database, logprob_model3)
biogeme_model3.bootstrap_samples=100
results_bootstrapping = biogeme_model3.estimate(run_bootstrap=True)

You have not defined a name for the model. The output files are named from the model name. The default is [biogemeModelDefaultName]
100%|█████████████████████████████████████████| 100/100 [02:56<00:00,  1.76s/it]


We obtain a sample of values for the parameters. We use then to calculate empirically (that is, using simulation), the 90% confidence intervals on the simulated quantities.

In [79]:
betas = biogeme_model3.freeBetaNames()
b = results_bootstrapping.getBetasForSensitivityAnalysis(betas)

The syntax "freeBetaNames" is deprecated and is replaced by the syntax "free_beta_names".


In [80]:
left, right = biosim.confidenceIntervals(b, 0.9)

In [81]:
left['Weighted walk'] = (
    left['Weight'] * 
    left['Prob. walk']
)
left['Weighted drive'] = (
    left['Weight'] * 
    left['Prob. drive']
)
left['Weighted pt'] = (
    left['Weight'] * 
    left['Prob. pt']
)
left['Weighted cycle'] = (
    left['Weight'] * 
    left['Prob. cycle']
)

In [82]:
left_market_share_walk = left['Weighted walk'].mean()
left_market_share_drive = left['Weighted drive'].mean()
left_market_share_pt = left['Weighted pt'].mean()
left_market_share_cycle = left['Weighted cycle'].mean()

In [92]:
right['Weighted walk'] = (
    right['Weight'] * 
    right['Prob. walk']
)
right['Weighted drive'] = (
    right['Weight'] * 
    right['Prob. drive']
)
right['Weighted pt'] = (
    right['Weight'] * 
    right['Prob. pt']
)
right['Weighted cycle'] = (
    right['Weight'] * 
    right['Prob. cycle']
)

In [93]:
right_market_share_walk = right['Weighted walk'].mean()
right_market_share_drive = right['Weighted drive'].mean()
right_market_share_pt = right['Weighted pt'].mean()
right_market_share_cycle = right['Weighted cycle'].mean()


In [94]:
print(
    f'Market share for walking: {100*market_share_walk:.1f}% '
    f'CI: ['
    f'{100*left_market_share_walk:.1f}%-'
    f'{100*right_market_share_walk:.1f}'
    f']'
)
print(
    f'Market share for driving: {100*market_share_drive:.1f}% '
    f'CI: ['
    f'{100*left_market_share_drive:.1f}%-'
    f'{100*right_market_share_drive:.1f}'
    f']'
)
print(
    f'Market share for public transportation: {100*market_share_pt:.1f}% '
    f'CI: ['
    f'{100*left_market_share_pt:.1f}%-'
    f'{100*right_market_share_pt:.1f}'
    f']'
)
print(
    f'Market share for cycling: {100*market_share_cycle:.1f}% '
    f'CI: ['
    f'{100*left_market_share_cycle:.1f}%-'
    f'{100*right_market_share_cycle:.1f}'
    f']'
)

Market share for walking: 17.8% CI: [16.6%-18.9]
Market share for driving: 43.0% CI: [40.7%-45.4]
Market share for public transportation: 36.2% CI: [34.1%-38.2]
Market share for cycling: 3.0% CI: [2.4%-3.7]


#### Comparing with the weighted market shares computed using the actual choices

Instead of using the predictive Model we will use the actual choice of the individual from the sample and the weights computed earlier to calculate the Markets shares.

In [86]:
LPMC['walk']= (LPMC.travel_mode==1)
LPMC['drive']= (LPMC.travel_mode==4)
LPMC['cycle']=(LPMC.travel_mode==2)
LPMC['pt']=(LPMC.travel_mode==3)

In [87]:
LPMC ['Weighted walk real'] = (
    LPMC ['Weight'] * 
    LPMC ['walk']
)
LPMC ['Weighted drive real'] = (
    LPMC ['Weight'] * 
    LPMC ['drive']
)
LPMC ['Weighted cycle real'] = (
    LPMC ['Weight'] * 
    LPMC ['cycle']
)
LPMC ['Weighted pt real'] = (
    LPMC ['Weight'] * 
    LPMC ['pt']
)

In [88]:
market_share_walk_real = (LPMC['Weighted walk real'].sum())/5000
print(f'Market share for walking from the sample: {100*market_share_walk_real:.1f}%')

market_share_cycle_real = (LPMC['Weighted cycle real'].sum())/5000
print(f'Market share for cycling from the sample: {100*market_share_cycle_real:.1f}%')

market_share_drive_real = (LPMC['Weighted drive real'].sum())/5000
print(f'Market share for driving from the sample: {100*market_share_drive_real:.1f}%')

market_share_pt_real = (LPMC['Weighted pt real'].sum())/5000
print(f'Market share for public transport from the sample: {100*market_share_pt_real:.1f}%')

Market share for walking from the sample: 17.9%
Market share for cycling from the sample: 3.1%
Market share for driving from the sample: 42.6%
Market share for public transport from the sample: 36.5%
